In [1]:
import pandas as pd
import cv2
import numpy as np
from matplotlib import pyplot as plt

affectnet_path = 'C:/Users/Lun/Documents/Study/MSBD/CSIT6000G - Machine Learning/9) Project/AffectNet/automatically_annotated.csv'
affectnet_path_2 = 'C:/Users/Lun/Documents/Study/MSBD/CSIT6000G - Machine Learning/9) Project/AffectNet/training.csv'

# AffectNet Internet image
affectnet_auto_data = pd.read_csv(affectnet_path)
# AffectNet Manual Annotated image
affectnet_manual_data = pd.read_csv(affectnet_path_2)

In [2]:
# Image Catelogue - AffectNet Internet image
affectnet_auto_data['expression'].value_counts(sort=False)

# 0: Neutral, 1: Happiness, 2: Sadness, 3: Surprise, 4: Fear, 5: Disgust, 6: Anger,7: Contempt, 8: None, 9: Uncertain, 10: No-Face

0     143142
1     246235
2      20854
3      17462
4       3799
5        890
6      28000
7          2
8       7259
9       1447
10     70517
Name: expression, dtype: int64

In [3]:
# Image Catelogue - AffectNet Manual Annotated image
affectnet_manual_data['expression'].value_counts(sort=False)

0      74874
1     134415
2      25459
3      14090
4       6378
5       3803
6      24882
7       3750
8      33088
9      11645
10     82415
Name: expression, dtype: int64

In [ ]:
import os
import shutil

# Due to limitation on disk size and network speed.  Only 1/10 images were downlaoded.
# From the image catelogue, we filter out the downloaded disgust images for our data supplement
affectnet_auto_data_10 = affectnet_auto_data.loc[affectnet_auto_data["subDirectory_filePath"].astype(str).str[:2]=='10']
affectnet_auto_data_10_disgust = affectnet_auto_data_10.loc[affectnet_auto_data["expression"]==5]

new =  affectnet_auto_data_10_disgust["subDirectory_filePath"].str.split("/", n = 1, expand = True) 
file_path = affectnet_auto_data_10_disgust["subDirectory_filePath"].values.tolist()
file_name = new[1].values.tolist()
affectnet_path_3 = 'C:/Users/Lun/Documents/Study/MSBD/CSIT6000G - Machine Learning/9) Project/AffectNet/Automatically_Annotated_Images/'
affectnet_path_4 = 'C:/Users/Lun/Documents/Study/MSBD/CSIT6000G - Machine Learning/9) Project/AffectNet/Manually_Annotated/disgust/'

# Copy all the disgust images into a separate folder for futher data pre-processing
for i in range(len(file_name)):
    full_path = affectnet_path_3+file_name[i]

    exists = os.path.isfile(full_path)
    if exists:
        shutil.copyfile(full_path, affectnet_path_4+file_name[i]) 
        print(affectnet_path_4+file_name[i])


In [ ]:
from keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
import os
import csv  

directory = 'C:/Users/Lun/Documents/Study/MSBD/CSIT6000G - Machine Learning/9) Project/AffectNet/Manually_Annotated/disgust/'

# For each disgust image
for filename in os.listdir(directory):

    img = image.load_img(directory+filename, grayscale=True, target_size=(48, 48))

    # load the image into 2D array
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    
    # Resize the image into 48 x 48
    x = np.array(x, 'float32')
    x = x.reshape([48, 48])
    
    # Data augmentation - Flip the image
    flip_1 = np.fliplr(x)
    
    # Serialize the image into 1D array
    x = x.reshape([1,2304])

    # Transform the array into a list
    x_list = x[0].tolist()
    x_str = " ".join(str(i) for i in x_list)

    # Append the additional disgust image into Kaggle FER2013 dataset for model training
    fields=[1,x_str,'Training']
    with open(r'C:/Users/Lun/PycharmProjects/Emotion_Detection/data/fer2013_augmented.csv', 'a') as f:
        writer = csv.writer(f)
        writer.writerow(fields)